<a href="https://colab.research.google.com/github/adarshr00711/Stock-price-prediction/blob/master/PX_kpi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#dta read
from azureml.core import Run
import argparse
import os
import pandas as pd
import numpy as np
import pyodbc
# dataset object from the run
parser=argparse.ArgumentParser('Data_read')
parser.add_argument('--output',type=str)
args = parser.parse_args()
output = args.output
df.head()
os.makedirs(output, exist_ok=True)
output_path = output + "/agg_input.csv"
print(output_path)
df.to_csv(output_path)

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:


 
#feature engineering

numeric_features = ['gap']
numeric_transformer = StandardScaler()

 


 
categorical_features = ['month_flight']
categorical_transformer = OneHotEncoder()

 


 
preprocessor = ColumnTransformer(
transformers=[
("num", numeric_transformer, numeric_features),
("cat", categorical_transformer, categorical_features)
]
)

 


 
numeric_features2 = ['total_sold']
preprocessor2 = ColumnTransformer(
transformers=[
("num", numeric_transformer, numeric_features2)
]
)

 


 

enc = preprocessor.fit_transform(X_train)

 


 
X_train_encoded = pd.DataFrame.sparse.from_spmatrix(enc)
X_test_encoded = pd.DataFrame.sparse.from_spmatrix(preprocessor.transform(X_test))
X_val_encoded = pd.DataFrame.sparse.from_spmatrix(preprocessor.transform(X_val))

 


 
enc2 = preprocessor2.fit_transform(y_train)

 


 
y_train_encoded = pd.DataFrame(enc2)
y_test_encoded = pd.DataFrame(preprocessor2.transform(y_test))

In [ ]:
from azureml.core import Run
import argparse
import os
import pandas as pd
import numpy as np
import pyodbc
#------------------------
run = Run.get_context()
parser=argparse.ArgumentParser('aggregation')
parser.add_argument('--input',type=str)
parser.add_argument('--output',type=str) 
args = parser.parse_args()
input1=args.input  #parse the 'aggregation' data refernce for input data location
output = args.output #parse the argument to get the file location
input_file = input1 + "/agg_input.csv"
df=pd.read_csv(input_file)
#------------------------
def area_allsl(df):
    #functions takes the preprocessed df as input
    #categorical df
    df1=df[['DimDateKey','FYMonthsAgo','UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2','RankCDName', 'CompensationBracket', 'CumlativeBadgeCountCategory', 'PromotionsCount', 'HireCount', 'UserDefAreaHierarchy1Change', 'HiresCampus', 'HiresExperienced', 'HiresExecutive', 'HiresIntern', 'HeadCount', 'Workforce']]
    #num df
    df2=df[['DimDateKey','FYMonthsAgo','UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2','RankCDName', 'GPSPctFavorableUtilizationCareerOpties', 'GPSPctFavorableUtilizationWorkLifeBalance', 'GPSPctFavorableUtilizationLearningOpties','effective_utilization','LeadScorePPEDD']]
    # # Numerical GroupBy
    # In[30]:
    df_grp_num=df2.drop(['UserDefSLHierarchy1','UserDefSLHierarchy2'],axis=1).groupby(['UserDefAreaHierarchy1','RankCDName','DimDateKey','FYMonthsAgo']).mean()
    df_grp_num=df_grp_num.reset_index()
    a=['All' for i in range(len(df_grp_num))]
    df_grp_num['UserDefSLHierarchy1']=a
    df_grp_num['UserDefSLHierarchy2']=a
    #Categorical GroupBy
    #can be use for all categorical variables to give a proportion
    def proportion(df):
        a=(df.value_counts()/len(df)).values
        b=(df.value_counts()/len(df)).index
        res = {b[i]: a[i] for i in range(len(b))}
        return(res)
    #should be used for binary categories in which denominator should be hirecount/headcount  
    def proportion_denom(df):
        a=(df[df.columns[0]].value_counts()/sum(df[df.columns[1]])).values
        b=(df[df.columns[0]].value_counts()/sum(df[df.columns[1]])).index
        if(len(b)>1):
            res = {b[i]:a[i] for i in range(len(b))}
        elif(0 in b):
            res={0:1,1:0}
        else:
            res={0:0,1:1}
        return(res)
    grp_cmp=df1.drop(['UserDefSLHierarchy1','UserDefSLHierarchy2'],axis=1).groupby(['UserDefAreaHierarchy1','RankCDName','DimDateKey','FYMonthsAgo'])
    cols2=['HiresCampus', 'HiresExperienced', 'HiresExecutive', 'HiresIntern']
    combined_df=grp_cmp[['HireCount','HeadCount']].apply(proportion_denom)
    combined_df=pd.DataFrame(combined_df).reset_index()
    a=[]
    b=[]
    for i in combined_df[0]:
        a.append(i[0])
        b.append(i[1])
    combined_df['HireCount_0']=a
    combined_df['HireCount_1']=b
    combined_df=combined_df.drop(0,axis=1)
    for j in cols2:
        d=grp_cmp[[j,'HireCount']].apply(proportion_denom)
        d=pd.DataFrame(d).reset_index()
        a=[]
        b=[]
        for i in d[0]:
            a.append(i[0])
            b.append(i[1])
        combined_df[j+'_0']=a
        combined_df[j+'_1']=b
    a=['All' for i in range(len(combined_df))]
    combined_df['UserDefSLHierarchy2']=a
    combined_df['UserDefSLHierarchy1']=a
#     combined_df.head()
    cols1=['CumlativeBadgeCountCategory', 'PromotionsCount', 'UserDefAreaHierarchy1Change', 'Workforce']
    combined_df2=grp_cmp['CompensationBracket'].apply(proportion)
    combined_df2=pd.DataFrame(combined_df2).reset_index()
    combined_df2['CompensationBracket_index']=combined_df2['level_4']
    combined_df2=combined_df2.drop('level_4',axis=1)
    for i in cols1:
        d=grp_cmp[i].apply(proportion)
        d=pd.DataFrame(d).reset_index()
combined_df2[i+'_index']=d['level_4']
        combined_df2[i]=d[i]
#     combined_df2.head()
    a=['All' for i in range(len(combined_df2))]
    combined_df2['UserDefSLHierarchy2']=a
    combined_df2['UserDefSLHierarchy1']=a
#     combined_df2.head()
    # combined_df.to_csv('combined_categorical.csv')
    a=(combined_df2.pivot_table('CompensationBracket',['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 
                                                   'UserDefSLHierarchy2', 'RankCDName', 'DimDateKey','FYMonthsAgo'],'CompensationBracket_index')).reset_index()
    cols=['CumlativeBadgeCountCategory', 'PromotionsCount',  'UserDefAreaHierarchy1Change', 'Workforce']
    for i in cols:
        b=(combined_df2.pivot_table(i,['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 
                                                   'UserDefSLHierarchy2', 'RankCDName', 'DimDateKey','FYMonthsAgo'],i+'_index')).reset_index()
        l=len(combined_df2[i+'_index'].value_counts())
        b=b[b.columns[-l:]]
        n=[]
        for j in b.columns:
            n.append(str(i)+'_'+str(j))
        b.columns=n
        a=pd.concat([a,b],axis=1)
#     a.head()#propotions dataframe without hirecount and headcount
    categorical_df=pd.merge(a,combined_df,on=['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2', 'RankCDName', 'DimDateKey', 'FYMonthsAgo'])
#     categorical_df.head()
    categorical_df.columns=['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2', 'RankCDName', 'DimDateKey', 'FYMonthsAgo', 'CompensationBracket_High', 'CompensationBracket_Low', 'CompensationBracket_Medium', 'CumlativeBadgeCountCategory_Badge(s) begun but not complete', 'CumlativeBadgeCountCategory_More than one badge complete', 'CumlativeBadgeCountCategory_No badges begun', 'CumlativeBadgeCountCategory_One badge complete', 'PromotionsCount_0.0', 'PromotionsCount_1.0', 'UserDefAreaHierarchy1Change_0.0', 'UserDefAreaHierarchy1Change_1.0', 'Workforce_0.0', 'Workforce_1.0', 'HireCount_0', 'HireCount_1', 'HiresCampus_0', 'HiresCampus_1', 'HiresExperienced_0', 'HiresExperienced_1', 'HiresExecutive_0', 'HiresExecutive_1', 'HiresIntern_0', 'HiresIntern_1']
    categorical_df=categorical_df.fillna(0)
    categorical_df=categorical_df.drop(['HireCount_0','HiresCampus_0','HiresExperienced_0','HiresExecutive_0',
                                        'PromotionsCount_0.0','UserDefAreaHierarchy1Change_0.0','HiresIntern_0'],axis=1)
#     categorical_df.head()
    # Merging columns
    # In[47]:
    # temp1=pd.merge(categorical_df,df_grp_num,on=['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2','RankCDName'])
    # temp1.head()
    temp2=pd.merge(categorical_df,df_grp_num,on=['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2','RankCDName','DimDateKey','FYMonthsAgo'])
#     temp2.head()
    temp2.columns=['ManagerialArea','ServiceLine','SubServiceLine','Rank','DimDateKey', 'FYMonthsAgo', '% in High Compensation Bracket', '% in Low Compensation Bracket', '% in Medium Compensation Bracket', '% Began Badge Journey', '% Completed More Than One Badge', '% Not Began Badge Journey', '% Completed Exactly One Badge ', '% Promoted in Last Year', '% Switched Service Lines', '% Not as Headcount (e.g. Contract)', '% Headcount (e.g. FTE)', '% New Hires among Employees', '% Campus Hires among New Hires', '% Experienced Hires among New Hires', '% Executive Hires among New Hires', '% Interns among New Hires', 'GPS: % Favorable of Career Opportunities', 'GPS: % Favorable of Work/Life Balance', 'GPS: % Favorable of Learning Opportunities', 'effective_utilization','Average PPEDDs LEAD Score']
    #  temp2.head()
    # temp2.to_csv('Area-All SL.csv')
    return(temp2)
df1,df2,df3,df4,df5,df6=area_allsl(df),area_sl_allsll(df),area_sl_ssl(df),allarea_allsl(df),allarea_sl_allsll(df),allarea_sl_ssl(df)
combdf=pd.concat([df1,df2,df3,df4,df5,df6],axis=0)
# combdf.head()
df_final=combdf[['ManagerialArea','ServiceLine','SubServiceLine','Rank','DimDateKey', 'FYMonthsAgo', '% in High Compensation Bracket', '% in Low Compensation Bracket', '% in Medium Compensation Bracket', '% Began Badge Journey', '% Completed More Than One Badge', '% Not Began Badge Journey', '% Completed Exactly One Badge ', '% Promoted in Last Year', '% Switched Service Lines', '% Not as Headcount (e.g. Contract)', '% Headcount (e.g. FTE)', '% New Hires among Employees', '% Campus Hires among New Hires', '% Experienced Hires among New Hires', '% Executive Hires among New Hires', '% Interns among New Hires', 'GPS: % Favorable of Career Opportunities', 'GPS: % Favorable of Work/Life Balance', 'GPS: % Favorable of Learning Opportunities', 'effective_utilization','Average PPEDDs LEAD Score']]
a=np.arange(0,len(df_final))
df_final.index=a
# df_final.head()
# making directory and writing to csv which is later used for modelling
os.makedirs(output, exist_ok=True)
output_path = output + "/model_input.csv"
print(output_path)
df_final.to_csv(output_path)
print(df_final.head())
run.complete()

In [ ]:
from azureml.pipeline.core import PipelineData,Pipeline
from azureml.pipeline.steps import PythonScriptStep
agg_input=PipelineData('agg_input',datastore=def_blob_store).as_dataset()
model_input=PipelineData('model_input',datastore=def_blob_store).as_dataset()
from azureml.pipeline.steps import PythonScriptStep
source_directory =cwd
#2-mlcomputeD12
#-mlwrkspaceSP02
# built-in step to run a Python Script on a compute target. It takes a script name and optionally other parameters like arguments for the script, compute target, inputs and outputs.
dataread = PythonScriptStep(
    name="Data_read",
    script_name="DataRead.py", 
    arguments=['--output',agg_input],
    outputs=[agg_input],
    source_directory=source_directory,
    compute_target=data_read_compute,
    runconfig=run_config)
agg=PythonScriptStep(name='aggregation',
                      script_name='aggregation.py',
                      arguments=['--input',agg_input,'--output',model_input],
                      inputs=[agg_input],
                      outputs=[model_input],
                      compute_target=data_read_compute,
                      runconfig=run_config,
                      source_directory=source_directory,
                      allow_reuse=True)
model=PythonScriptStep(name='modelling',
                      script_name='modelling.py',
                      arguments=['--input',model_input],
                      inputs=[model_input],
#                       outputs=[model_input],
                      compute_target=aml_compute,
                      runconfig=run_config,
                      source_directory=source_directory,
                      allow_reuse=True)
pipeline=Pipeline(workspace=ws,steps=[dataread,agg,model])
pipeline.validate() #validating pipelines for circular dependencies and parameter checks etc.
from azureml.core import Experiment
# create an ML experiment
exp = Experiment(workspace=ws, name='60ML')
from datetime import datetime
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule,TimeZone
timenow = datetime.now().strftime('%m-%d-%Y-%H-%M')
pipeline_name = timenow + "-Pipeline"
print(pipeline_name)
published_pipeline1 = pipeline.publish(
name=pipeline_name,
description=pipeline_name)
print("Newly published pipeline id: {} ".format(published_pipeline1.id))
pub_pipeline_id=published_pipeline1.id
import datetime
start_time = datetime.datetime(year=2020, month=10, day=12, hour=16,minute=18,second=0)
recurrence = ScheduleRecurrence(frequency="Hour", interval=3,start_time=start_time,time_zone=TimeZone.IndiaStandardTime)
schedule = Schedule.create(workspace=ws, name="PX360_schedule",pipeline_id=pub_pipeline_id,experiment_name='PX360_schedule_exp',
                        recurrence=recurrence,
                        wait_for_provisioning=True,
                        description="Schedule Run")
# You may want to make sure that the schedule is provisioned properly
# before making any further changes to the schedule
print("Created schedule with id: {} ".format(schedule.id))

In [ ]:
from azureml.pipeline.core import PipelineData,Pipeline
from azureml.pipeline.steps import PythonScriptStep
agg_input=PipelineData('agg_input',datastore=def_blob_store).as_dataset()
model_input=PipelineData('model_input',datastore=def_blob_store).as_dataset()
from azureml.pipeline.steps import PythonScriptStep
source_directory =cwd
#2-mlcomputeD12
#-mlwrkspaceSP02
# built-in step to run a Python Script on a compute target. It takes a script name and optionally other parameters like arguments for the script, compute target, inputs and outputs.
dataread = PythonScriptStep(
    name="Data_read",
    script_name="DataRead.py", 
    arguments=['--output',agg_input],
    outputs=[agg_input],
    source_directory=source_directory,
    compute_target=data_read_compute,
    runconfig=run_config)
agg=PythonScriptStep(name='aggregation',
                      script_name='aggregation.py',
                      arguments=['--input',agg_input,'--output',model_input],
                      inputs=[agg_input],
                      outputs=[model_input],
                      compute_target=data_read_compute,
                      runconfig=run_config,
                      source_directory=source_directory,
                      allow_reuse=True)
model=PythonScriptStep(name='modelling',
                      script_name='modelling.py',
                      arguments=['--input',model_input],
                      inputs=[model_input],
#                       outputs=[model_input],
                      compute_target=aml_compute,
                      runconfig=run_config,
                      source_directory=source_directory,
                      allow_reuse=True)
pipeline=Pipeline(workspace=ws,steps=[dataread,agg,model])
pipeline.validate() #validating pipelines for circular dependencies and parameter checks etc.
from azureml.core import Experiment
# create an ML experiment
exp = Experiment(workspace=ws, name='60ML')
from datetime import datetime
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule,TimeZone
timenow = datetime.now().strftime('%m-%d-%Y-%H-%M')
pipeline_name = timenow + "-Pipeline"
print(pipeline_name)
published_pipeline1 = pipeline.publish(
name=pipeline_name,
description=pipeline_name)
print("Newly published pipeline id: {} ".format(published_pipeline1.id))
pub_pipeline_id=published_pipeline1.id
import datetime
start_time = datetime.datetime(year=2020, month=10, day=12, hour=16,minute=18,second=0)
recurrence = ScheduleRecurrence(frequency="Hour", interval=3,start_time=start_time,time_zone=TimeZone.IndiaStandardTime)
schedule = Schedule.create(workspace=ws, name="PX360_schedule",pipeline_id=pub_pipeline_id,experiment_name='PX360_schedule_exp',
                        recurrence=recurrence,
                        wait_for_provisioning=True,
                        description="Schedule Run")
# You may want to make sure that the schedule is provisioned properly
# before making any further changes to the schedule
print("Created schedule with id: {} ".format(schedule.id))
#data agg
from azureml.core import Run
import argparse
import os
import pandas as pd
import numpy as np
import pyodbc
#------------------------
run = Run.get_context()
parser=argparse.ArgumentParser('aggregation')
parser.add_argument('--input',type=str)
parser.add_argument('--output',type=str) 
args = parser.parse_args()
input1=args.input  #parse the 'aggregation' data refernce for input data location
output = args.output #parse the argument to get the file location
input_file = input1 + "/agg_input.csv"
df=pd.read_csv(input_file)
#------------------------
def area_allsl(df):
    #functions takes the preprocessed df as input
    #categorical df
    df1=df[['DimDateKey','FYMonthsAgo','UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2','RankCDName', 'CompensationBracket', 'CumlativeBadgeCountCategory', 'PromotionsCount', 'HireCount', 'UserDefAreaHierarchy1Change', 'HiresCampus', 'HiresExperienced', 'HiresExecutive', 'HiresIntern', 'HeadCount', 'Workforce']]
    #num df
    df2=df[['DimDateKey','FYMonthsAgo','UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2','RankCDName', 'GPSPctFavorableUtilizationCareerOpties', 'GPSPctFavorableUtilizationWorkLifeBalance', 'GPSPctFavorableUtilizationLearningOpties','effective_utilization','LeadScorePPEDD']]
    # # Numerical GroupBy
    # In[30]:
    df_grp_num=df2.drop(['UserDefSLHierarchy1','UserDefSLHierarchy2'],axis=1).groupby(['UserDefAreaHierarchy1','RankCDName','DimDateKey','FYMonthsAgo']).mean()
    df_grp_num=df_grp_num.reset_index()
    a=['All' for i in range(len(df_grp_num))]
    df_grp_num['UserDefSLHierarchy1']=a
    df_grp_num['UserDefSLHierarchy2']=a
    #Categorical GroupBy
    #can be use for all categorical variables to give a proportion
    def proportion(df):
        a=(df.value_counts()/len(df)).values
        b=(df.value_counts()/len(df)).index
        res = {b[i]: a[i] for i in range(len(b))}
        return(res)
    #should be used for binary categories in which denominator should be hirecount/headcount  
    def proportion_denom(df):
        a=(df[df.columns[0]].value_counts()/sum(df[df.columns[1]])).values
        b=(df[df.columns[0]].value_counts()/sum(df[df.columns[1]])).index
        if(len(b)>1):
            res = {b[i]:a[i] for i in range(len(b))}
        elif(0 in b):
            res={0:1,1:0}
        else:
            res={0:0,1:1}
        return(res)
    grp_cmp=df1.drop(['UserDefSLHierarchy1','UserDefSLHierarchy2'],axis=1).groupby(['UserDefAreaHierarchy1','RankCDName','DimDateKey','FYMonthsAgo'])
    cols2=['HiresCampus', 'HiresExperienced', 'HiresExecutive', 'HiresIntern']
    combined_df=grp_cmp[['HireCount','HeadCount']].apply(proportion_denom)
    combined_df=pd.DataFrame(combined_df).reset_index()
    a=[]
    b=[]
    for i in combined_df[0]:
        a.append(i[0])
        b.append(i[1])
    combined_df['HireCount_0']=a
    combined_df['HireCount_1']=b
    combined_df=combined_df.drop(0,axis=1)
    for j in cols2:
        d=grp_cmp[[j,'HireCount']].apply(proportion_denom)
        d=pd.DataFrame(d).reset_index()
        a=[]
        b=[]
        for i in d[0]:
            a.append(i[0])
            b.append(i[1])
        combined_df[j+'_0']=a
        combined_df[j+'_1']=b
    a=['All' for i in range(len(combined_df))]
    combined_df['UserDefSLHierarchy2']=a
    combined_df['UserDefSLHierarchy1']=a
#     combined_df.head()
    cols1=['CumlativeBadgeCountCategory', 'PromotionsCount', 'UserDefAreaHierarchy1Change', 'Workforce']
    combined_df2=grp_cmp['CompensationBracket'].apply(proportion)
    combined_df2=pd.DataFrame(combined_df2).reset_index()
    combined_df2['CompensationBracket_index']=combined_df2['level_4']
    combined_df2=combined_df2.drop('level_4',axis=1)
    for i in cols1:
        d=grp_cmp[i].apply(proportion)
        d=pd.DataFrame(d).reset_index()
combined_df2[i+'_index']=d['level_4']
        combined_df2[i]=d[i]
#     combined_df2.head()
    a=['All' for i in range(len(combined_df2))]
    combined_df2['UserDefSLHierarchy2']=a
    combined_df2['UserDefSLHierarchy1']=a
#     combined_df2.head()
    # combined_df.to_csv('combined_categorical.csv')
    a=(combined_df2.pivot_table('CompensationBracket',['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 
                                                   'UserDefSLHierarchy2', 'RankCDName', 'DimDateKey','FYMonthsAgo'],'CompensationBracket_index')).reset_index()
    cols=['CumlativeBadgeCountCategory', 'PromotionsCount',  'UserDefAreaHierarchy1Change', 'Workforce']
    for i in cols:
        b=(combined_df2.pivot_table(i,['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 
                                                   'UserDefSLHierarchy2', 'RankCDName', 'DimDateKey','FYMonthsAgo'],i+'_index')).reset_index()
        l=len(combined_df2[i+'_index'].value_counts())
        b=b[b.columns[-l:]]
        n=[]
        for j in b.columns:
            n.append(str(i)+'_'+str(j))
        b.columns=n
        a=pd.concat([a,b],axis=1)
#     a.head()#propotions dataframe without hirecount and headcount
    categorical_df=pd.merge(a,combined_df,on=['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2', 'RankCDName', 'DimDateKey', 'FYMonthsAgo'])
#     categorical_df.head()
    categorical_df.columns=['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2', 'RankCDName', 'DimDateKey', 'FYMonthsAgo', 'CompensationBracket_High', 'CompensationBracket_Low', 'CompensationBracket_Medium', 'CumlativeBadgeCountCategory_Badge(s) begun but not complete', 'CumlativeBadgeCountCategory_More than one badge complete', 'CumlativeBadgeCountCategory_No badges begun', 'CumlativeBadgeCountCategory_One badge complete', 'PromotionsCount_0.0', 'PromotionsCount_1.0', 'UserDefAreaHierarchy1Change_0.0', 'UserDefAreaHierarchy1Change_1.0', 'Workforce_0.0', 'Workforce_1.0', 'HireCount_0', 'HireCount_1', 'HiresCampus_0', 'HiresCampus_1', 'HiresExperienced_0', 'HiresExperienced_1', 'HiresExecutive_0', 'HiresExecutive_1', 'HiresIntern_0', 'HiresIntern_1']
    categorical_df=categorical_df.fillna(0)
    categorical_df=categorical_df.drop(['HireCount_0','HiresCampus_0','HiresExperienced_0','HiresExecutive_0',
                                        'PromotionsCount_0.0','UserDefAreaHierarchy1Change_0.0','HiresIntern_0'],axis=1)
#     categorical_df.head()
    # Merging columns
    # In[47]:
    # temp1=pd.merge(categorical_df,df_grp_num,on=['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2','RankCDName'])
    # temp1.head()
    temp2=pd.merge(categorical_df,df_grp_num,on=['UserDefAreaHierarchy1', 'UserDefSLHierarchy1', 'UserDefSLHierarchy2','RankCDName','DimDateKey','FYMonthsAgo'])
#     temp2.head()
    temp2.columns=['ManagerialArea','ServiceLine','SubServiceLine','Rank','DimDateKey', 'FYMonthsAgo', '% in High Compensation Bracket', '% in Low Compensation Bracket', '% in Medium Compensation Bracket', '% Began Badge Journey', '% Completed More Than One Badge', '% Not Began Badge Journey', '% Completed Exactly One Badge ', '% Promoted in Last Year', '% Switched Service Lines', '% Not as Headcount (e.g. Contract)', '% Headcount (e.g. FTE)', '% New Hires among Employees', '% Campus Hires among New Hires', '% Experienced Hires among New Hires', '% Executive Hires among New Hires', '% Interns among New Hires', 'GPS: % Favorable of Career Opportunities', 'GPS: % Favorable of Work/Life Balance', 'GPS: % Favorable of Learning Opportunities', 'effective_utilization','Average PPEDDs LEAD Score']
    #  temp2.head()
    # temp2.to_csv('Area-All SL.csv')
    return(temp2)
df1,df2,df3,df4,df5,df6=area_allsl(df),area_sl_allsll(df),area_sl_ssl(df),allarea_allsl(df),allarea_sl_allsll(df),allarea_sl_ssl(df)
combdf=pd.concat([df1,df2,df3,df4,df5,df6],axis=0)
# combdf.head()
df_final=combdf[['ManagerialArea','ServiceLine','SubServiceLine','Rank','DimDateKey', 'FYMonthsAgo', '% in High Compensation Bracket', '% in Low Compensation Bracket', '% in Medium Compensation Bracket', '% Began Badge Journey', '% Completed More Than One Badge', '% Not Began Badge Journey', '% Completed Exactly One Badge ', '% Promoted in Last Year', '% Switched Service Lines', '% Not as Headcount (e.g. Contract)', '% Headcount (e.g. FTE)', '% New Hires among Employees', '% Campus Hires among New Hires', '% Experienced Hires among New Hires', '% Executive Hires among New Hires', '% Interns among New Hires', 'GPS: % Favorable of Career Opportunities', 'GPS: % Favorable of Work/Life Balance', 'GPS: % Favorable of Learning Opportunities', 'effective_utilization','Average PPEDDs LEAD Score']]
a=np.arange(0,len(df_final))
df_final.index=a
# df_final.head()
# making directory and writing to csv which is later used for modelling
os.makedirs(output, exist_ok=True)
output_path = output + "/model_input.csv"
print(output_path)
df_final.to_csv(output_path)
print(df_final.head())
run.complete()
#dta read
from azureml.core import Run
import argparse
import os
import pandas as pd
import numpy as np
import pyodbc
# dataset object from the run
parser=argparse.ArgumentParser('Data_read')
parser.add_argument('--output',type=str)
args = parser.parse_args()
output = args.output
df.head()
 Save the dataframe
os.makedirs(output, exist_ok=True)
output_path = output + "/agg_input.csv"
print(output_path)
df.to_csv(output_path)


Adarsh
j

Adarsh
px modelling
from azureml.core import Run
import argparse
import os
import pandas as pd
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.model_selection import train_test_split,KFold,GridSearchCV
from sklearn import metrics
from sklearn import model_selection
from sklearn.impute import KNNImputer
import pyodbc
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from scipy.stats import pearsonr
from datetime import date
# dataset object from the run
run = Run.get_context()
parser=argparse.ArgumentParser('aggregation')
parser.add_argument('--input',type=str)
# parser.add_argument('--output',type=str)

args = parser.parse_args()
input1=args.input
# output = args.output
pd.set_option('display.max_rows', 500)
input_file = input1 + "/model_input.csv"
df_fact=pd.read_csv(input_file,index_col=0)
cxn=
df_metric=pd.read_sql('SELECT  * FROM EY_PX360_PRD.MetricInformation',cnxn)
df_metric=df_metric.drop(['UpdatedDateTime'],axis=1)
df_metric['Rank']=df_metric['RankCDName']
df_metric=df_metric.drop('RankCDName',axis=1)
df_metric=df_metric.replace('$null$',np.nan)

df_metric['MetricValue']=df_metric['MetricValue'].astype(float)

df_metric=df_metric.drop(['ManagerialRegion','Country','UnitOfMetric','FrontEndReady','Headcount'],axis=1)
df_metric_pivot=(df_metric.pivot_table('MetricValue',['FYMonthsAgo', 'ServiceLine', 'SubServiceLine', 'ManagerialArea', 
                                         'Rank'],'MetricName')).reset_index()

metrics=df_metric['MetricName'].unique()
def automate(area,sl,subsl,Target):
    
    #filtering
    global df_fact
    df_fact=df_fact[(df_fact['Rank']!='55-Administrative Lead')&(df_fact['Rank']!='56-Administrative Advanced')&
        (df_fact['Rank']!='57-Administrative Intermediate')&(df_fact['Rank']!='58-Administrative Entry')]
    df_filter_fact=df_fact[(df_fact['ManagerialArea']==area)&(df_fact['ServiceLine']==sl)
                      &(df_fact['SubServiceLine']==subsl)]
    df_filter_metric=df_metric_pivot[(df_metric_pivot['ManagerialArea']==area)&(df_metric_pivot['ServiceLine']==sl)
                                &(df_metric_pivot['SubServiceLine']==subsl)]
    #merging
    cols=['% in High Compensation Bracket', '% in Low Compensation Bracket', '% in Medium Compensation Bracket', 
          '% Began Badge Journey', '% Completed More Than One Badge', 
          '% Not Began Badge Journey', '% Completed Exactly One Badge ', '% Promoted in Last Year', 
          '% Switched Service Lines', '% Not as Headcount (e.g. Contract)', '% Headcount (e.g. FTE)', '% New Hires among Employees', '% Campus Hires among New Hires', '% Experienced Hires among New Hires', '% Executive Hires among New Hires', '% Interns among New Hires']
    for i in cols:
        df_filter_fact[i]=df_filter_fact[i].fillna(0)
    temp1=pd.merge(df_filter_fact,df_filter_metric,on=['ManagerialArea', 'FYMonthsAgo', 'ServiceLine', 'SubServiceLine','Rank'],how='inner')
    #preprocessing
    rej_logs=""
    temp1=temp1.drop('effective_utilization',axis=1)
    init_shape=temp1.shape[1]
    temp1=temp1[temp1[Target].isna()==False]
    temp1=temp1[temp1.columns[(temp1.isnull().sum()<(0.6*len(temp1))).values]]
    cur_shape=temp1.shape[1]
    if(init_shape>cur_shape):
        rej_logs_null="Discarded Columns having more than 60% null values"
    else:
        rej_logs_null=""
    if(len(temp1)>0):
        temp1=temp1.drop_duplicates()
    if(len(temp1)>10):        
        #splitting Data
        X=temp1.drop(['ManagerialArea','ServiceLine','SubServiceLine',Target,'DimDateKey','FYMonthsAgo'],axis=1)
        y=temp1[Target]
        if(Target=='Eff. Utilization'):
            X=X[(X['Rank']=='44-Staff/Assistant')|(X['Rank']=='42-Senior')
               |(X['Rank']=='32-Manager')|(X['Rank']=='21-Senior Manager')|(X['Rank']=='11-Partner/Principal')|
               (X['Rank']=='13-Executive Director')]
        X['Rank']=X['Rank'].map({'44-Staff/Assistant':1,'42-Senior':2,'32-Manager':3,'21-Senior Manager':4,'11-Partner/Principal':6,
                          '13-Executive Director':5,'51-Intern (CS)':0,'53-Intern (CBS)':0,'05-Non Emp CBS':0,'04-Non Emp CS':0,
                                    '66-Associate':1,'65-Senior Associate':2,'64-Supv Associate':3,'63-Assistant Director':4
                                    ,'62-Associate Director':5,'61-Director':6})
        #scaling the data
        sc=StandardScaler()
        for i in X.columns:
            X[i]=sc.fit_transform(X[[i]])
        #knn imputation
        imputer=KNNImputer()
        imputer.fit(X)
        X_KNN=imputer.transform(X)
        X_knn=pd.DataFrame(X_KNN)
        X_knn.columns=X.columns
        ridge_model=Ridge(normalize=True)
        #feature selection using pearsonr correlation test
        X_corr=X_knn.copy()
        r_val,pval,col=[],[],[]
        for i in X_corr.columns:
            r, p = pearsonr(X_corr[i], y)
            r_val.append(r)
            pval.append(p)
        crdf=pd.DataFrame(columns=["Coeff","pvalue","feature"])
        crdf["Coeff"]=r_val
        crdf["pvalue"]=pval
        crdf["feature"]=list(X_corr.columns)
        features_pearson=list(crdf[crdf["pvalue"]<0.05]['feature'])
        X_knn=X_knn[features_pearson]
        #putting a cut off of features having a high  correlation among themselves
        b=pd.DataFrame(np.tril(X_knn.corr(), k=-1))
b.columns=X_knn.columns
        b.index=X_knn.columns
        X_knn=X_knn[b.index[(b[b>0.9]).any(axis=1)==False]]
        #modelling
        if(len(X_knn.columns)>2):
            print(Target)
            print('combination:',area,'-',sl,'-',subsl)   
            valid_comb.append((area,sl,subsl))
            global num_models
            num_models=num_models+1
            from sklearn.model_selection import GridSearchCV
            params_ridge={'alpha':np.arange(0.01,1,0.01)}
            GS_m2=GridSearchCV(ridge_model,params_ridge,cv=3,scoring='r2')
            GS_m2.fit(X_knn,y)
            parameter=GS_m2.best_params_['alpha']
            ridge_model=Ridge(alpha=parameter,normalize=True)
            models=[]
            models.append(('Ridge',ridge_model))
            xtrain,xtest,ytrain,ytest=train_test_split(X_knn,y,test_size=0.3,random_state=0)
            results=[]
            names=[]
            for name,model in models:
                kfold=model_selection.KFold(n_splits=3,shuffle=True,random_state=0)
                cv_result=model_selection.cross_val_score(model,X_knn,y,cv=kfold,scoring='neg_mean_squared_error')
                cv_result_acc=model_selection.cross_val_score(model,X_knn,y,cv=kfold,scoring='r2')
                results.append(np.sqrt(np.abs(cv_result)))
                names.append(name)
                BE=np.mean(np.sqrt(np.abs(cv_result)))
                VE=np.var(np.sqrt(np.abs(cv_result)))
                print(name)
                model.fit(xtrain,ytrain)
                crossval=np.mean(cv_result_acc)
                Training_acc=model.score(xtrain,ytrain)
                Testing_acc=model.score(xtest,ytest)
                print("\t\tCrossValAccuracy: ",crossval)
                print('\t\tTraining Accuracy: ',Training_acc)
                print('\t\tTesting ACcuracy: ',Testing_acc)
                print('\t\tBias Error: ',BE)
                print('\t\tVariance Error: ',VE)
                ypred=model.predict(xtrain)
                mse=((ypred-ytrain)**2).mean()
                rmse=mse**0.5
                
                return (name,model,xtrain,ytrain,crossval,Training_acc,Testing_acc,BE,VE,rmse,len(X_knn))
        else:
            rej_logs="Failed Statistical & Peason tests" + " " +rej_logs_null
            rejected_comb.append((area,sl,subsl,rej_logs))
            return (0,0,0,0,0,0,0,0,0,0,0)
    else:
#         print('No Data Exist')
        rej_logs="Insufficient/Inappropriate Data for Modelling"
        rejected_comb.append((area,sl,subsl,rej_logs))
        return (0,0,0,0,0,0,0,0,0,0,0)
for metric in metrics:
    ar=df_fact['ManagerialArea'].unique()
    S=df_fact['ServiceLine'].unique()
    SS=df_fact['SubServiceLine'].unique()
    flag=False
    valid_comb=[]
    rejected_comb=[]
    num_models=0
    for i in ar:
        for j in S:
            for k in SS:
                Name,Model,xtrain,ytrain,crossval,Training_acc,Testing_acc,BE,VE,rmse,length=automate(i,j,k,metric)
                if(Name!=0 and flag==False):
                    a=[i for p in range(len(xtrain.columns))]
                    Model.fit(xtrain,ytrain)
                    model_coef=pd.DataFrame({'Features':xtrain.columns,'Coefficients':Model.coef_,'Area':a,'ServiceLine':b,
                                             'SubServiceLine':c,'Cross Validation Accuracy':d,'Training accuracy':e,
                                             'Testing Accuracy':f,'Bias Error':g,'Variance Error':h,'RMSE':z,
                                             'Number of Data points':y})
                    flag=True
    f(len(valid_comb)>0):
        valid_comb=pd.DataFrame(valid_comb,columns=['Area','ServiceLine','SubServiceLine'])
        valid_comb.to_csv("valid combinations "+metric+".csv")
        num=[num_models for p in range(len(model_coef))]
        model_coef['Number of Models']=num
        model_coef.to_csv(metric+'.csv')
#write
cols=df.columns
df['DimDateKey']=DimDateKey
cols=cols.insert(0,'DimDateKey')
df=df[np.array(cols)]
df.columns=['DimDateKey', 'Features_Number', 'Features', 'Coefficients', 'Area',
       'ServiceLine', 'SubServiceLine', 'Cross_Validation_Accuracy',
       'Training_accuracy', 'Testing_Accuracy', 'Bias_Error', 'Variance_Error',
       'RMSE', 'Number_of_Data_Points', 'Number_of_Models', 'Metric']
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
for index, row in df.iterrows():
    cursor.execute("""INSERT INTO EY_PX360_PRD.ML_Output (DimDateKey, Features_Number, Features, Coefficients, Area,
       ServiceLine, SubServiceLine, Cross_Validation_Accuracy,
       Training_accuracy, Testing_Accuracy, Bias_Error, Variance_Error,
       RMSE, Number_of_Data_Points, Number_of_Models, Metric) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""", row.DimDateKey,
                   row.Features_Number, row.Features, row.Coefficients,row.Area,row.ServiceLine,row.SubServiceLine,
                   row.Cross_Validation_Accuracy,row.Training_accuracy,row.Testing_Accuracy,row.Bias_Error,
                   row.Variance_Error,row.RMSE,row.Number_of_Data_Points,row.Number_of_Models,row.Metric)
cnxn.commit()
cursor.close()





#!/usr/bin/env python
# coding: utf-8
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.model_selection import train_test_split,KFold,GridSearchCV
from sklearn import metrics
from sklearn import model_selection
from scipy.stats import pearsonr
import sys
import warnings
warnings.simplefilter("ignore")
from pandas_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df.head()
mapd.head()
mapd=mapd[['Employee Engagement.1', 'Employee Engagement.2', 'Employee Engagement.3', 'Employee Engagement.4', 'Employee Engagement.5', 'Ethics.1', 'Ethics.2', 'Ethics.3', 'Ethics.4', 'Career Development.1', 'Career Development.2', 'Supervisor/Employee Interaction.1', 'Supervisor/Employee Interaction.2', 'Supervisor/Employee Interaction.3', 'Employee Empowerment Index.1', 'Employee Empowerment Index.2', 'Employee Empowerment Index.3', 'Employee Empowerment Index.4', 'Employee Empowerment Index.5', 'Diversity and Inclusion.1', 'Diversity and Inclusion.2', 'Diversity and Inclusion.3', 'Diversity and Inclusion.4', 'Culture.1', 'Culture.2', 'Culture.3']]
mapd
df.head(2)
#mapping dict
namedict={}
for i in df.columns:
    if i in list(mapd.iloc[0]):
        coname=((mapd == i).idxmax(axis=1)[0])
        namedict[i]=coname
# namedict
initdf=df.copy()
pxidf = df.rename(columns=namedict)
pxidf.head(10)
att_orgdf=pd.read_excel(r'data\Attrition_at_OrgLevel_2020.xlsx')
att_orgdf.head() #emp separated
# 50000005-sample
att_orgdf[att_orgdf["Org Unit ID"]==50000012]
att_orgdf_active=pd.read_excel(r'Attrition_Active_Emp.xlsx')
att_orgdf_active=att_orgdf_active[att_orgdf_active["Year"]==2020.0]
att_orgdf_active[att_orgdf_active["Org Unit ID"]==50000012] # sample org unit
print(att_orgdf.shape)
att_orgdf_active.columns
att_df=pd.read_excel(r'Active_Seprated_Emp_Orglevel_2020.xlsx')
att_df.head(10)
len(att_df["Org Unit ID"].unique())
pxidf=pxidf.dropna()
pxidf['Org Unit Number']=pxidf['Org Unit Number'].astype(int)
pxidf=pxidf.drop(['Group', 'Group.1', 'Respondents', 'Career Development', 'Culture', 
                  'Diversity and Inclusion', 'Employee Empowerment Index', 'Employee Engagement', 'Ethics',
                  'Supervisor/Employee Interaction','Recipients', 'Response Rate', 'Hierarchy Path'],axis=1)
pxidf.head()
findf=pd.merge(pxidf,att_df,left_on="Org Unit Number",right_on="Org Unit ID",how='inner')
findf.head()
print(findf.shape)
findf=findf.drop(['Org Unit Number', 'Org Unit' ,'Emp Seperated','Org Unit ID','ActiveEmployees'],axis=1)
final_df=findf.copy()
final_df.head()
def model_dim(X,y):
#     xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=1)
    RR=Ridge()
    LR=LinearRegression()
    models=[]
    models.append(('Linear Regression',LR))
#     models.append(("Ridge Regression",RR))
#     print("DIMENSION:",kpi)
    dic={}
    results=[]
    names=[]
    for name,model in models:
        kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=0)
        cv_result=model_selection.cross_val_score(model,X,y,cv=kfold,scoring='neg_mean_squared_error')
        cv_result_acc=model_selection.cross_val_score(model,X,y,cv=kfold,scoring='r2')
        results.append(np.sqrt(np.abs(cv_result)))
        names.append(name)
        BE=np.mean(np.sqrt(np.abs(cv_result)))
        VE=np.var(np.sqrt(np.abs(cv_result)))
#         print(name)
        model.fit(X,y)
#         print('\t\tTraining Accuracy: ',model.score(xtrain,ytrain))
#         print('\t\tTesting Accuracy: ',model.score(xtest,ytest))
#         print('\t\tBias Error: ',BE)
#         print('\t\tVariance Error: ',VE)
#         ypred=model.predict(xtest)
#         rmse=np.sqrt(metrics.mean_squared_error(ytest,ypred))
#         print('RMSE: ',rmse)
        r2=model.score(X,y)
        print('R2: ',r2)
        adj_r2 = (1 - (1 - r2) * ((X.shape[0] - 1) / 
                  (X.shape[0] - X.shape[1] - 1)))
        print("adjusted R2",adj_r2)
#         dic=[kpi,r2,adj_r2]
        coeff=pd.Series(model.coef_,X.columns).sort_values(ascending=False)
    return(coeff,r2,adj_r2)
kpilist=['Career Development', 'Culture', 'Diversity and Inclusion', 'Employee Empowerment Index', 'Employee Engagement', 'Ethics', 'Supervisor/Employee Interaction']
y=final_df['AttritionRate']
datalist=[]
dim_df=pd.DataFrame()
for kpi in kpilist:
    filter_col = [col for col in final_df if col.startswith(kpi)]
    filtered_df=final_df[filter_col]
    filtered_df[kpi] = filtered_df.mean(axis=1)
    dim_df=pd.concat([filtered_df,dim_df],axis=1)
# resdf = pd.DataFrame(datalist, columns = ['Dimension', 'R2 Square','Adjusted R square'])
# print(resdf)
dim_df=dim_df[kpilist]
dim_df.head()
final_df=pd.concat([dim_df,y],axis=1)
final_df.corr()["AttritionRate"].sort_values(ascending=False)
from statsmodels.stats.outliers_influence import variance_inflation_factor
# the independent variables set 
X = final_df
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                        for i in range(len(X.columns))] 
print(vif_data)
final_df.corr().head()
final_df.head()
dim_df.head()
# scaling
sc=StandardScaler()
for i in final_df.columns:
    final_df[i]=sc.fit_transform(final_df[[i]])
y=final_df['AttritionRate']
data,r2,adjr2=model_dim(dim_df,y)
data 
# In[118]:
data=data.to_frame().reset_index()
data.columns=["Dimension","Weightage"]
data["R2"]=r2
data["Adj R2"]=adjr2
data["DataPoints"]=final_df.shape[0]
data.to_csv("Results/Attrition_Results.csv")
data #pre
filter_col = [col for col in final_df if col.startswith("Diversity and Inclusion")]
filtered_df=final_df[filter_col]
# the independent variables set 
X = filtered_df
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                        for i in range(len(X.columns))] 
print(vif_data)
filtered_df.corr().head()
filter_col = [col for col in final_df if col.startswith("Culture")]
filtered_df=final_df[filter_col]
# the independent variables set 
X = filtered_df
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = X.columns 
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(X.values, i) 
                        for i in range(len(X.columns))] 
print(vif_data)
filtered_df.corr().head()
final_df.shape
len(final_df[final_df["AttritionRate"]==0.0])
final_df.columns
import seaborn as sns
sns.kdeplot(y)
(final_df.corr()['AttritionRate']).sort_values(ascending=False)
# In[70]:
def model_dim(X,y,kpi):
#     xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=1)
    RR=Ridge()
    LR=LinearRegression()
    models=[]
    models.append(('Linear Regression',LR))
    models.append(("Ridge Regression",RR))
    print("DIMENSION:",kpi)
    dic={}
    results=[]
    names=[]
    for name,model in models:
        kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=0)
        cv_result=model_selection.cross_val_score(model,X,y,cv=kfold,scoring='neg_mean_squared_error')
        cv_result_acc=model_selection.cross_val_score(model,X,y,cv=kfold,scoring='r2')
        results.append(np.sqrt(np.abs(cv_result)))
        names.append(name)
        BE=np.mean(np.sqrt(np.abs(cv_result)))
        VE=np.var(np.sqrt(np.abs(cv_result)))
        print(name)
model.fit(X,y)
#         print('\t\tTraining Accuracy: ',model.score(xtrain,ytrain))
#         print('\t\tTesting Accuracy: ',model.score(xtest,ytest))
#         print('\t\tBias Error: ',BE)
#         print('\t\tVariance Error: ',VE)
#         ypred=model.predict(xtest)
#         rmse=np.sqrt(metrics.mean_squared_error(ytest,ypred))
#         print('RMSE: ',rmse)
        r2=model.score(X,y)
        print('R2: ',r2)
        adj_r2 = (1 - (1 - r2) * ((X.shape[0] - 1) / 
                  (X.shape[0] - X.shape[1] - 1)))
        print("adjusted R2",adj_r2)
        dic=[kpi,r2,adj_r2]
#         LR.fit(xtrain,ytrain)
#         LR.coef_
#         pd.Series(LR.coef_,X.columns).sort_values(ascending=False)
    return(dic)
kpilist=['Career Development', 'Culture', 'Diversity and Inclusion', 'Employee Empowerment Index', 'Employee Engagement', 'Ethics', 'Supervisor/Employee Interaction']
y=final_df['AttritionRate']
datalist=[]
for kpi in kpilist:
    filter_col = [col for col in final_df if col.startswith(kpi)]
    filtered_df=final_df[filter_col]
    kpidict={}
    data=model_dim(filtered_df,y,kpi)
    datalist.append(data)
    print("--------\n")
resdf = pd.DataFrame(datalist, columns = ['Dimension', 'R2 Square','Adjusted R square'])
print(resdf)
resdf.to_csv("Attrition_Reg_Results.csv")
final_df.head()
final_df.drop(["AttritionRate"],axis=1,inplace=True)
def attbin(att):
    if(att>0.0):
        s=1
    else:
        s=0
    return s
final_df["Attbin"]=final_df.apply(lambda x:attbin(x["AttritionRate"]),axis=1)
final_df.head()
# In[118]:
filtered_df['mean'] = filtered_df.mean(axis=1)
filtered_df.head()
# In[121]:
import numpy as np
import statsmodels.api as sm
# kpilist=['Employee Engagement']
y=final_df['Attbin']
kpilist=['Career Development', 'Culture', 'Diversity and Inclusion', 'Employee Empowerment Index', 'Employee Engagement', 'Ethics', 'Supervisor/Employee Interaction']
datalist=[]
for kpi in kpilist:
    filter_col = [col for col in final_df if col.startswith(kpi)]
    filtered_df=final_df[filter_col]
    filtered_df['mean'] = filtered_df.mean(axis=1)
    filtered_df=filtered_df[['mean']]
    model = sm.Logit(y, filtered_df)
    result = model.fit(method='newton')
    print(kpi)
    print(result.summary())
def model_dim(X,y,kpi):
    xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=1)
    LR=LinearRegression()
    RR=Ridge()
    models=[]
#     models.append(('Linear Regression',LR))
    models.append(("Ridge Regression",RR))
    dic={}
    results=[]
    names=[]
    for name,model in models:
        kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=0)
        cv_result=model_selection.cross_val_score(model,X,y,cv=kfold,scoring='neg_mean_squared_error')
        cv_result_acc=model_selection.cross_val_score(model,X,y,cv=kfold,scoring='r2')
        results.append(np.sqrt(np.abs(cv_result)))
        names.append(name)
        BE=np.mean(np.sqrt(np.abs(cv_result)))
        VE=np.var(np.sqrt(np.abs(cv_result)))
        print(name)
        model.fit(xtrain,ytrain)
        print('\t\tTraining Accuracy: ',model.score(xtrain,ytrain))
        print('\t\tTesting Accuracy: ',model.score(xtest,ytest))
#         print('\t\tBias Error: ',BE)
#         print('\t\tVariance Error: ',VE)
        ypred=model.predict(xtest)
        rmse=np.sqrt(metrics.mean_squared_error(ytest,ypred))
        print("DIMENSION:",kpi)
        r2=model.score(X,y)
        print('R2: ',r2)
        adj_r2 = (1 - (1 - r2) * ((X.shape[0] - 1) / 
                  (X.shape[0] - X.shape[1] - 1)))
        print("adjusted R2",adj_r2)
        dic=[kpi,r2,adj_r2]
#         LR.fit(xtrain,ytrain)
#         LR.coef_
#         pd.Series(LR.coef_,X.columns).sort_values(ascending=False)
    return(dic)
#     LR.fit(xtrain,ytrain)
#     LR.coef_
#     pd.Series(LR.coef_,X.columns).sort_values(ascending=False)
# In[78]:
datalist=[]
for kpi in kpilist:
    filter_col = [col for col in final_df if col.startswith(kpi)]
    filtered_df=final_df[filter_col]
    kpidict={}
    data=model_dim(filtered_df,y,kpi)
    datalist.append(data)
    print("--------\n")
resdf = pd.DataFrame(datalist, columns = ['Dimension', 'R2 Square','Adjusted R square'])
print(resdf)
# In[37]:
def model_dim(X,y,kpi):
    xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=1)
    LR=LinearRegression()
    RR=Ridge()
    models=[]
#     models.append(('Linear Regression',LR))
    models.append(("Ridge Regression",RR))
    print("DIMENSION:",kpi)
    dic={}
    results=[]
    names=[]
    for name,model in models:
        kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=0)
        cv_result=model_selection.cross_val_score(model,X,y,cv=kfold,scoring='neg_mean_squared_error')
        cv_result_acc=model_selection.cross_val_score(model,X,y,cv=kfold,scoring='r2')
        results.append(np.sqrt(np.abs(cv_result)))
        names.append(name)
        BE=np.mean(np.sqrt(np.abs(cv_result)))
        VE=np.var(np.sqrt(np.abs(cv_result)))
        print(name)
        model.fit(xtrain,ytrain)
        print('\t\tTraining Accuracy: ',model.score(xtrain,ytrain))
        print('\t\tTesting Accuracy: ',model.score(xtest,ytest))
#         print('\t\tBias Error: ',BE)
#         print('\t\tVariance Error: ',VE)
        ypred=model.predict(xtest)
        rmse=np.sqrt(metrics.mean_squared_error(ytest,ypred))
        print('RMSE: ',rmse)
        r2=model.score(xtrain,ytrain)
        print('R2: ',r2)
        adj_r2 = (1 - (1 - r2) * ((xtrain.shape[0] - 1) / 
                  (xtrain.shape[0] - xtrain.shape[1] - 1)))
        print("adjusted R2",adj_r2)
        dic={"rmse":rmse,"R2 square":r2,"Adjusted R2":adj_r2}
    return(dic)
#     LR.fit(xtrain,ytrain)
#     LR.coef_
#     pd.Series(LR.coef_,X.columns).sort_values(ascending=False)
kpilist=['Career Development', 'Culture', 'Diversity and Inclusion', 'Employee Empowerment Index', 'Employee Engagement', 'Ethics', 'Supervisor/Employee Interaction']
y=final_df['AttritionRate']
for kpi in kpilist:
    filter_col = [col for col in final_df if col.startswith(kpi)]
    filtered_df=final_df[filter_col]
    kpidict={}
    kpidict[kpi]=model_dim(filtered_df,y,kpi)
    print(kpidict)
    print("----------------\n")
X=final_df.drop('AttritionRate',axis=1)
y=final_df['AttritionRate']
X.head()
kpilist=['Employee Engagement']
for kpi in kpilist:
    filter_col = [col for col in final_df if col.startswith(kpi)]
    filtered_df=final_df[filter_col]
X=filtered_df
X.shape
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=1)
LR=LinearRegression()
models=[]
models.append(('Linear Regression',LR))
results=[]
names=[]
for name,model in models:
    kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=0)
    cv_result=model_selection.cross_val_score(model,X,y,cv=kfold,scoring='neg_mean_squared_error')
    cv_result_acc=model_selection.cross_val_score(model,X,y,cv=kfold,scoring='r2')
    results.append(np.sqrt(np.abs(cv_result)))
    names.append(name)
    BE=np.mean(np.sqrt(np.abs(cv_result)))
    VE=np.var(np.sqrt(np.abs(cv_result)))
    print(name)
    model.fit(xtrain,ytrain)
    print('\t\tTraining Accuracy: ',model.score(xtrain,ytrain))
    print('\t\tTesting Accuracy: ',model.score(xtest,ytest))
    print('\t\tBias Error: ',BE)
    print('\t\tVariance Error: ',VE)
LR.fit(xtrain,ytrain)
LR.coef_
pd.Series(LR.coef_,X.columns).sort_values(ascending=False)
ypred=model.predict(xtest)
rmse=np.sqrt(metrics.mean_squared_error(ytest,ypred))
print('RMSE: ',rmse)
r2=model.score(xtrain,ytrain)
print('R2: ',r2)
adj_r2 = (1 - (1 - r2) * ((xtrain.shape[0] - 1) / 
          (xtrain.shape[0] - xtrain.shape[1] - 1)))
print("adjR2",adj_r2)